## Auto mode

This is for testing propouses

In [1]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import QGC, ConfigGazebo, ConfigQGC, Gazebo, NoneVisualizer, Simulator

clean()

## Simulation Configuration

In [2]:
mission_name = "square"

gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 0., 0., 0.),
    (15., 0., 0., 0),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),
    (5., -20., 0., 0.),
])
n_uavs = len(base_homes)
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin= gra_origin)

for path,home,c in zip(base_paths,base_homes,colors):
    qgc_config.add(base_path=path,base_home=home,color=c)
qgc_config.show()

## Save the missions

In [5]:
qgc_config.save_missions('square')

## Visualization Parameters

In [6]:
novis=NoneVisualizer()

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [7]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz],
	n_uavs=n_uavs, # take this out
	terminals=False,
	verbose=1,
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1]})

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 3 launched (PID 61950)
🚀 ArduPilot SITL vehicle 1 launched (PID 61946)
🚀 ArduPilot SITL vehicle 2 launched (PID 61948)
🚀 ArduPilot SITL vehicle 4 launched (PID 61951)
🚀 ArduPilot SITL vehicle 5 launched (PID 61953)
🚀 UAV logic for vehicle 3 launched (PID 61956)
🚀 UAV logic for vehicle 1 launched (PID 61957)
🚀 UAV logic for vehicle 4 launched (PID 61959)
🚀 UAV logic for vehicle 2 launched (PID 61958)
🚀 Proxy for vehicle 3 launched (PID 61963)
🔗 UAV logic 3 is connected to Ardupilot SITL vehicle 3
🚀 UAV logic for vehicle 5 launched (PID 61961)
🚀 Proxy for vehicle 1 launched (PID 61966)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
🚀 Proxy for vehicle 2 launched (PID 61970)
🔗 UAV logic 2 is connected to Ardupilot SITL vehicle 2
🚀 Proxy for vehicle 4 launched (PID 61969)
🔗 UAV logic 4 is connected to Ardupilot SITL vehicle 4
🚀 Proxy for vehicle 5 launched (PID 61975)
🔗 UAV logic 5 is connect

bind port 5830 for SERIAL0
SERIAL0 on TCP port 5830
bind port 5820 for SERIAL0
SERIAL0 on TCP port 5820
bind port 5780 for SERIAL0
SERIAL0 on TCP port 5780
bind port 5810 for SERIAL0
SERIAL0 on TCP port 5810
bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


System id: 2
System id: 5
Connection on serial port 5830
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 4
Connection on serial port 5820
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 2
Connection on serial port 5780

🚀 Starting Proxy 2
System id: 3
Connection on serial port 5810
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/aut

Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.



🚀 Starting Proxy 5

🚀 Starting Proxy 4

🚀 Starting Proxy 3


Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.


Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/def

bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures
bind port 5832 for SERIAL1
SERIAL1 on TCP port 5832
bind port 5833 for SERIAL2
SERIAL2 on TCP port 5833
bind port 5822 for SERIAL1
SERIAL1 on TCP port 5822
bind port 5823 for SERIAL2
SERIAL2 on TCP port 5823
validate_structures:528: Validating structures
bind port 5812 for SERIAL1
SERIAL1 on TCP port 5812
bind port 5813 for SERIAL2
SERIAL2 on TCP port 5813
bind port 5782 for SERIAL1
SERIAL1 on TCP port 5782
bind port 5783 for SERIAL2
SERIAL2 on TCP port 5783
validate_structures:528: Validating structures
validate_structures:528: Validating structures
validate_structures:528: Validating structures


## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 3: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 2: ▶️ Step Started: clear uav missions
Vehicle 3: ▶️ Step Started: clear uav missions
🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 5: ✅ Step Done: clear uav missions
Vehicle 4: ✅ Step Done: clear uav missions
Vehicle 3: ✅ Step Done: clear uav missions
Vehicle 2: ✅ Step Done: clear uav missions
Vehicle 1: ✅ 7 waypoints read.
Vehicle 5: ✅ 7 waypoints read.
Vehicle 4: ✅ 7 waypoints read.
Vehicle 3: ✅ 7 waypoints read.
Vehicle 2: ✅ 7 waypoints read.
Vehicle 1: ✅ Sent mission item 0
Vehicle 5: ✅ Sent mission item 0
Vehicle 4: ✅ Sent mission item 0
Vehicle 3: ✅ Sent mission item 0
Vehicle 2: ✅ Sent mission item 0
Vehicle 1: ✅ Sent mission item 1
Vehicle 5: ✅ Sent mission item 1
Vehicle 4: ✅ Sent mission item 1
Vehicle 